https://github.com/ricsinaruto/NeuralChatbots-DataFiltering/blob/master/code/utils/filtering_demo.ipynb

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import math
from collections import Counter
from tqdm.notebook import tqdm
from pathlib import Path

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None) # don't truncate cell contents

from df_utils import *

from datasets import load_dataset



In [2]:
# # run demo on daily dialog
# dataset = load_dataset('daily_dialog', split='train')
# dataset

# sources, targets = [], []
# for dialog in tqdm(dataset['dialog']):
#     for i in range(len(dialog)-1):
#         sources.append(dialog[i].strip())
#         targets.append(dialog[i+1].strip())
# #         print(i)

# print(len(sources))
# print(len(targets))

In [3]:
# run demo on review response data
data = Path('../data/hotel/500k/trip_hotels.pkl')
df = pd.read_pickle(data)
df.drop(columns=['Unnamed: 0'], inplace=True)
print(df.columns)
print(len(df))


Index(['domain', 'rating', 'review_author', 'response_author', 'review_clean',
       'response_clean', 'sentiment', 'db_internal_id', 'establishment',
       'trip_id', 'trip_url', 'country', 'split',
       'score:review_response_length_ratio', 'score:response_sentence_length',
       'score:genericness_semantic_avg', 'score:genericness_length_ratio',
       'score:review_response_wmd', 'rrgen_id', 'trip_review_url',
       'trip_review_id'],
      dtype='object')
500000


In [4]:
sources = df.review_clean.to_list()
targets = df.response_clean.to_list()


In [5]:
targets[0]

'Dear Lucy J, Thank you for sharing your experience with the Trip Advisor community. I am thrilled you enjoyed the amenities and services during your visit to Uptown. Again, thank you for taking the time to write such a lovely review about the hotel. We look forward to welcoming you back in the near future. Sincerely, David Rothwell General Manager'

In [6]:
#@title Path to the source lines and target lines.
# sPath = "sources.txt"  #@param {type:"string"}
# tPath = "targets.txt"  #@param {type:"string"}

In [7]:
def build_distribution(data):
    # Read data and store it in a list.
    if isinstance(data, str):
        with open(data) as file:
            clean_sentences = [' '.join(s.split()) for s in file]
    elif isinstance(data, list):
        clean_sentences = data
    
    # Build a dict of unique sentences storing indexes to the original list.
    sentence_dict = {}
    for i, sentence in tqdm(enumerate(clean_sentences)):
        sentence_dict[sentence] = sentence_dict.get(sentence, [])
        sentence_dict[sentence].append(i)
      
    return sentence_dict, clean_sentences

source_distro, sources = build_distribution(sources)
target_distro, targets = build_distribution(targets)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [9]:
print(len(source_distro))
print(type(source_distro))
print(list(source_distro.keys())[:3])
# print(source_distro['Can you do push-ups ?'])

499996
<class 'dict'>
['Fantastic! ---SEP--- We were treated like royalty during our stay. We actually live in Charlotte but stayed here during a two day event uptown in Charlotte. Staff incredible detail oriented and friendly. Superior service everywhere we looked! Prices are high but you get what you okay for. It was truly a pleasure to stay at such a high quality hotel. Would recommend to everyone!', "Convenient ---SEP--- The hotel is custom-made for a traveller in transit or anyone who wants to avoid the hassle of catching a flight at Heathrow's Terminal 4 in good time. A tunnel links the hotel to the terminal. From your room, you simply check out and drag your luggage through the tunnel to the departure hall. No hassles. The hotel is clean and the staff are quite friendly and helpful. Next door is the Holiday Inn which shares common elevator with Crowne Plaza. You can have the best of two worlds by also sampling the offerings in the hotel next door. Nice place!", "Authentic house,

In [42]:
print(len(target_distro))
print(type(target_distro))
print(list(target_distro.items())[:3])


482758
<class 'dict'>
[('Dear Lucy J, Thank you for sharing your experience with the Trip Advisor community. I am thrilled you enjoyed the amenities and services during your visit to Uptown. Again, thank you for taking the time to write such a lovely review about the hotel. We look forward to welcoming you back in the near future. Sincerely, David Rothwell General Manager', [0]), ('Dear OYEWOLE O, Thank you for your kind feedback and for choosing The New Crowne Plaza at London Heathrow T4. We are delighted to understand that you found our hotel to be convenient for your journey through London Heathrow Airport. We will share your most positive comments with our dedicated team. Thank you once again and we look forward to welcoming you in the future. Warm Regards, Andrew Brown. General Manager Crowne Plaza London Heathrow T4.', [1]), ("Jeff, Thank you for taking the time to write such a full review of the inn. It's great that your group got to experience both our new Clark House suites as

In [16]:
# from IPython.core.debugger import set_trace

def get_entropies(distro, pairs):
    entropies = {}

    for sentence, indices in tqdm(list(distro.items())):
        # Build a distribution for the current sentence, based on the targets.
        distribution = Counter([pairs[i] for i in indices]) # count of unique sources for a given target
        num_elements = len(indices) # count of total sources for a given target

        # Calculate entropy.
        entropy = 0
        for frequency in distribution.values():
            probability = frequency / num_elements
            entropy += probability * math.log(probability, 2)
        entropies[sentence] = -entropy

#     print(entropies)
    return entropies

source_entropies = get_entropies(source_distro, targets)
target_entropies = get_entropies(target_distro, sources)

  0%|          | 0/499996 [00:00<?, ?it/s]

  0%|          | 0/482758 [00:00<?, ?it/s]

In [49]:
0.5 * math.log(3, 2)

0.7924812503605781

In [61]:

sentence, indices = list(target_distro.items())[191]
distribution = Counter([sources[i] for i in indices])
# print(distribution)
print(len(distribution))
print(len(indices))
num_elements = len(indices)

entropy = 0
for frequency in distribution.values():
    probability = frequency / num_elements
    entropy += probability * math.log(probability, 2)
print(-entropy)


18
18
4.169925001442313


In [17]:
print(len(source_entropies))
print(len(target_entropies))

499996
482758


In [21]:
list(source_entropies.items())[:4]

[('Fantastic! ---SEP--- We were treated like royalty during our stay. We actually live in Charlotte but stayed here during a two day event uptown in Charlotte. Staff incredible detail oriented and friendly. Superior service everywhere we looked! Prices are high but you get what you okay for. It was truly a pleasure to stay at such a high quality hotel. Would recommend to everyone!',
  -0.0),
 ("Convenient ---SEP--- The hotel is custom-made for a traveller in transit or anyone who wants to avoid the hassle of catching a flight at Heathrow's Terminal 4 in good time. A tunnel links the hotel to the terminal. From your room, you simply check out and drag your luggage through the tunnel to the departure hall. No hassles. The hotel is clean and the staff are quite friendly and helpful. Next door is the Holiday Inn which shares common elevator with Crowne Plaza. You can have the best of two worlds by also sampling the offerings in the hotel next door. Nice place!",
  -0.0),
 ("Authentic house

In [26]:
# sum(target_entropies.values())

def inspect_entropy_scores(entropy_dict, max_items=10):
    cnt = 0
    for text, entropy in entropy_dict.items():
        if entropy > 0.0:
            print(text, entropy)
            cnt += 1
        if cnt == max_items:
            return None

inspect_entropy_scores(source_entropies)
inspect_entropy_scores(target_entropies)

Great property, even better staff! ---SEP--- This review will detail the extraordinary staff we experienced at both Le Meridien and Sheraton Hotel in Charlotte. The bottom line is that they are two excellent properties with a great style, close to the convention center, and our preference in the downtown Charlotte area. Now here are the details of my weeklong stay in downtown Charlotte, where we were fortunate enough to stay at both Le Meridien and Sheraton Hotels. This was a business trip for myself, but my wife was able to join me. The property, the excellent services, and professional staff made it feel more like a vacation. The atmosphere in the lobby and the guest rooms of Le Meridien is swanky and gives the feel and look of the modern world traveler. The seating in the lobby is inviting and stylish and provides a great place to meet up or chat with fellow guests. Andrew, on check-in, was courteous and professional. Even after we checked in he called to make sure the room was suit

In [33]:
threshold = 1.0
cnt = 0

tgt_ent = []
src_ent = []

for source, target in zip(sources, targets):
    # Here we can put whatever combination of the entropy thresholds.
    # if source_entropies[source] > threshold and target_entropies[target] > threshold:
    # Casky et al., 2019 report that identity applied to target entropies is best
    if target_entropies[target] > threshold:
#         print('SRC', source_entropies[source], source)
#         print('TGT', target_entropies[target], target)
        cnt += 1
print(cnt)

17289


In [34]:
df['src_entropy_identity'] = df['review_clean'].apply(lambda x: source_entropies[x])
df['tgt_entropy_identity'] = df['response_clean'].apply(lambda x: target_entropies[x])

In [35]:
df.head()

,domain,rating,review_author,response_author,review_clean,response_clean,sentiment,db_internal_id,establishment,trip_id,trip_url,country,split,score:review_response_length_ratio,score:response_sentence_length,score:genericness_semantic_avg,score:genericness_length_ratio,score:review_response_wmd,rrgen_id,trip_review_url,trip_review_id,src_entropy_identity,tgt_entropy_identity
0,Hotel,5,Lucy J,RCCharlotte,Fantastic! ---SEP--- We were treated like royalty during our stay. We actually live in Charlotte but stayed here during a two day event uptown in Charlotte. Staff incredible detail oriented and friendly. Superior service everywhere we looked! Prices are high but you get what you okay for. It was truly a pleasure to stay at such a high quality hotel. Would recommend to everyone!,"Dear Lucy J, Thank you for sharing your experience with the Trip Advisor community. I am thrilled you enjoyed the amenities and services during your visit to Uptown. Again, thank you for taking the time to write such a lovely review about the hotel. We look forward to welcoming you back in the near future. Sincerely, David Rothwell General Manager",3 -1 3,126080,"The Ritz-Carlton, Charlotte",1510383,https://www.tripadvisor.com/Hotel_Review-g49022-or530-d1510383-Reviews-The_Ritz_Carlton_Charlotte-Charlotte_North_Carolina.html,US,train,1.600000,5,0.879500,0.400000,4.637085,2329375,https://www.tripadvisor.com/ShowUserReviews-g49022-d1510383-r202056667-The_Ritz_Carlton_Charlotte-Charlotte_North_Carolina.html,202056667,-0.0,-0.0
1,Hotel,4,OYEWOLE O,CPHIEXManagementTeam,"Convenient ---SEP--- The hotel is custom-made for a traveller in transit or anyone who wants to avoid the hassle of catching a flight at Heathrow's Terminal 4 in good time. A tunnel links the hotel to the terminal. From your room, you simply check out and drag your luggage through the tunnel to the departure hall. No hassles. The hotel is clean and the staff are quite friendly and helpful. Next door is the Holiday Inn which shares common elevator with Crowne Plaza. You can have the best of two worlds by also sampling the offerings in the hotel next door. Nice place!","Dear OYEWOLE O, Thank you for your kind feedback and for choosing The New Crowne Plaza at London Heathrow T4. We are delighted to understand that you found our hotel to be convenient for your journey through London Heathrow Airport. We will share your most positive comments with our dedicated team. Thank you once again and we look forward to welcoming you in the future. Warm Regards, Andrew Brown. General Manager Crowne Plaza London Heathrow T4.",2 -2 2,147558,Crowne Plaza London Heathrow T4,14151392,https://www.tripadvisor.co.uk/Hotel_Review-g528813-or495-d14151392-Reviews-Crowne_Plaza_London_Heathrow_T4-Hounslow_Greater_London_England.html,UK,train,1.333333,6,0.829168,0.166667,4.310637,2770320,https://www.tripadvisor.co.uk/ShowUserReviews-g528813-d14151392-r643739102-Crowne_Plaza_London_Heathrow_T4-Hounslow_Greater_London_England.html,643739102,-0.0,-0.0
2,Hotel,4,Jeff M,fairholminnkeeper,"Authentic house, authentic people ---SEP--- Stayed for two nights, my wife and I - her sister and husband who are from Vancouver. all of us approx 60. He wanted an authentic historical setting, plus king bed, etc. We have done a lot of B & B's in Maine over the years so less of an issue for us. I called about a month ahead and they only had one King Bedroom left in the main house, but they had just redone their Coachhouse on the same property so inlaws in main house, my wife and I in Coach house. King Room was the real thing, king bed, propane fireplace, spacious. only modern was bathroom which was redone but with fixtures with a traditional look. very nice. Our 'room' was a one-bedroom apt, very modern looking, open kitchen (fully equiped) king bed, 2 TVs. could have stayed a month and been quite comfortable. Breakfast was two choices which varied each morning (i.e. eggs benedict), plus a yoghurt with fruit and granola, and baked goods. Ve

In [62]:
# result of filtering using naive identity method from Csaky et al., 2019
# filters on 5% of the data
df_filt = df[df['tgt_entropy_identity'] < 1.0]
print((len(df_filt) / len(df)) * 100)



95.687
